## Agile Hardware Design
***
# Testing

## Prof. Scott Beamer
### sbeamer@ucsc.edu

## [CSE 293](https://classes.soe.ucsc.edu/cse293/Winter22/)

## Plan for Today

* Testing overview
* Testing a combinational unit
* Tidying up with ScalaTest
* Testing a Decoupled (stateful) unit

## Loading The Chisel Library Into a Notebook

In [ ]:
val path = System.getProperty("user.dir") + "/../resource/chisel_deps.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

In [ ]:
import chisel3._
import chisel3.util._
import chisel3.tester._
import chisel3.tester.RawTester.test
import org.scalatest._

## Why Test?

### Who wants your hardware if it doesn't work?

### How do you prove to yourself it works?

### How do you prove to others it works?

### How do you even develop it?

## Goals of Today's Testing Lecture

### Develop techniques/abstractions to improve testing productivity

### View testing as an _integral_ part of development (not just verification)

### Learn more sophisticated ways to test in Chisel

## 3 Main Components of Testing

### 1) How do you generate test cases?
* Human-generated - best for simple cases or edge cases
* Synthetically generated - exhaustive or (directed) random

### 2) How do you know what is correct response to test?
* Human-generated - brittle and best to avoid after initial bootstrap
* Model-generated - highly preferable, but need to also test model

### 3) How do you simulate/execute/script test?
* In general - consider needs of flexibility, portability, speed
* _Today_: simulate with _treadle_, execute with _ChiselTest_, organize with _ScalaTest_

## Considerations for Designing Tests

### What do you need to get started?
* Close the loop early in order to do _test-driven development (TDD)_

### What is _coverage_ needed for this problem?
* Consider what you will need to test, and how be sure you covered it

### Should you treat design under test as _opaque_ or _clear_?
* Both!
* Users see _opaque_ module, so should properly implement specified interface
* Knowing implementation (_clear_) can help focus tests on likely edge cases

## Testing Advice

### Get humans out of the loop
  * Humans should help make tests, but not perform them
  * Print statements & waveforms are for debugging but not testing

### Random may not yield great coverage
  * In large test space, may have low probability of reaching interesting corner case
  * May want to _seed_ to get reproducibility

### Assertions are helpful, but do not replace need for testing
  * Tests don't just ensure consistent state, but also provide test stimuli
  * Assertions best for catching issues early (in simulation time) before they silently cause problems later
    * Consider adding them after debugging subtle bug

## When to Use Testing

* Helpful in many places and not just final verification
  * Initial development
  * Continuous integration running in background
  * Working with others (i.e. checking external contributions)
  * Design space exploration

* Consider testing early in process and design for it
  * Consider design abstractions and module boundaries to ease testing
  * Combinational modules can be easier to test, so place state elements deliberately

## Testing in Chisel

* Are generators harder to test?
  * Yes, but can parameterize test generation too!
  * Can amortize test development over all instances (produced by generator)
* [ChiselTest](https://www.chisel-lang.org/chiseltest/)
  * Can write testbenches directly in Scala
  * Runs as a Scala program that communicates with simulation of design
  * Upcoming Chisel library for testing (we have already been using it)
* Simulation options
  * [Treadle](https://www.chisel-lang.org/treadle/) - default FIRRTL simulator, implemented directly in Scala
    * Default, easy to get going and fastest for small designs
  * [Verilator](https://www.veripool.org/wiki/verilator) - fast open-source Verilog simulation, can talk to ChiselTest
    * Inter-process communication and ChiselTest can slow down overall
  * Other simulators - can simulate Verilog from Chisel, but won't be able to talk back to ChiselTest

## Testing a Combinational Component

* _Stateless_ (combinational) modules are easier to test since each test/cycle is independent

* _Consider:_
  * range of possible inputs
  * range of generator parameters
  * parameters' impact on input space

* If input space is sufficiently small, may be able to _exhaustively_ test
  * May be able to make sufficiently small by constraining parameters

## Combo. Example - Sign & Magnitude Add - Module Implementation

In [ ]:
class SignMag(n: Int) extends Bundle {
    val sign = Bool()
    val magn = UInt(n.W)
}

class SignMagAdd(n: Int) extends Module {
    val io = IO(new Bundle {
        val in0 = Input(new SignMag(n))
        val in1 = Input(new SignMag(n))
        val out = Output(new SignMag(n))
    })
    when (io.in0.sign === io.in1.sign) {
        io.out.sign := io.in0.sign
        io.out.magn := io.in0.magn + io.in1.magn
    } .elsewhen (io.in0.magn > io.in1.magn) {
        io.out.sign := io.in0.sign
        io.out.magn := io.in0.magn - io.in1.magn
    } otherwise {
        io.out.sign := io.in1.sign
        io.out.magn := io.in1.magn - io.in0.magn
    }
}

## Combo. Example - Sign & Magnitude Add - First Test

In [ ]:
test(new SignMagAdd(4)) { c =>
    c.io.in0.sign.poke(false.B)
    c.io.in0.magn.poke(1.U)
    c.io.in1.sign.poke(false.B)
    c.io.in1.magn.poke(2.U)
    c.io.out.sign.expect(false.B)
    c.io.out.magn.expect(3.U)
}

## Combo. Example - Sign & Magnitude Add - Make a Model

* Use Scala to generate the desired behavior
  * Can simply produce right output, or be full on class
* Be sure to model truncating/wrapping effects of data widths

In [ ]:
def modelAdd(a: Int, b: Int, n: Int): Int = {
    require(n > 0)
    require(n < 32)
    val mask = (1 << n) - 1
    val sum = a + b
    if (sum < 0) -((-sum) & mask)
    else sum & mask
}

modelAdd(4,4,4)

## Combo. Example - Sign & Magnitude Add - Automate Interaction

In [ ]:
def testAdd(a: Int, b: Int, n: Int, c: SignMagAdd, verbose: Boolean=true) {
    c.io.in0.sign.poke((a<0).B)
    c.io.in0.magn.poke(math.abs(a).U)
    c.io.in1.sign.poke((b<0).B)
    c.io.in1.magn.poke(math.abs(b).U)
    val outSignStr = if (c.io.out.sign.peek().litToBoolean) "-" else ""
    val outMag = c.io.out.magn.peek().litValue
    if (verbose)
        println(s"  in: $a + $b  out: $outSignStr$outMag")
    if (modelAdd(a,b,n) != 0)
        c.io.out.sign.expect((modelAdd(a,b,n) < 0).B)  // what is buggy here?
    c.io.out.magn.expect(math.abs(modelAdd(a,b,n)).U)
}

test(new SignMagAdd(4)) { c =>
    testAdd(2,3,4,c)
    testAdd(-1,5,4,c)
    testAdd(1,-1,4,c)
}

## Combo. Example - Sign & Magnitude Add - Test Exhaustively

In [ ]:
def testAll(n: Int) {
    val maxVal = (1<<n) - 1
    test(new SignMagAdd(n)) { c =>
        for (a <- -maxVal to maxVal) {
            for (b <- -maxVal to maxVal) {
                testAdd(a,b,n,c,false)
            }
        }
    }
}
testAll(2)

## Combo. Example - Sign & Magnitude Add - Random Test

In [ ]:
def testRandomAdd(n: Int, c: SignMagAdd) {
    def genInput() = {
        val limit = 1 << n
        val magn = scala.util.Random.nextInt(limit)
        val neg = scala.util.Random.nextBoolean
        if (neg) -magn else magn
    }
    testAdd(genInput(), genInput(), n, c)
}

def testRandomly(n: Int, numTrials: Int) {
    test(new SignMagAdd(n)) { c =>
        for (t <- 0 until numTrials)
            testRandomAdd(n,c)
    }
}

testRandomly(4, 5)

## ScalaTest

* Helpful [library](https://www.scalatest.org) to organize and group tests

* `sbt` is aware of it
  * Running `test` automatically runs all ScalaTests it can find
  * Can also use `testOnly package.class` to only test `package.class`

* ChiselTest can interoperate with it, and we have been using it in the homework already

## Combo. Example - Sign & Magnitude Add - with ScalaTest

In [ ]:
class SignMagAddTest(n: Int) extends FlatSpec with ChiselScalatestTester {
    behavior of s"SignMagAdd($n)"
    it should "1 + 2 = 3" in {
        test(new SignMagAdd(n)) { c =>
            testAdd(1,2,n,c)
        }
    }
    it should "1 - 1 = 0" in {
        test(new SignMagAdd(n)) { c =>
            testAdd(1,-1,n,c)
        }
    }
}

(new SignMagAddTest(4)).execute()

## Combo. Example - Sign & Magnitude Add - Bundle Literals

* Experimental [feature](https://github.com/ucb-bar/chisel-testers2/blob/master/src/test/scala/chiseltest/tests/BundleLiteralsSpec.scala) to specify a Bundle all at once

In [ ]:
import chisel3.experimental.BundleLiterals._

test(new SignMagAdd(4)) { c =>
    val b0 = chiselTypeOf(c.io.in0).Lit(_.sign->false.B, _.magn->2.U)
    val b1 = (new SignMag(4)).Lit(_.sign->false.B, _.magn->2.U)
    val s  = chiselTypeOf(c.io.out).Lit(_.sign->false.B, _.magn->4.U)
    c.io.in0.poke(b0)
    c.io.in1.poke(b1)
    c.io.out.expect(s)
}

## Seq. Example - Queue - Intro

* Testing stateful things is more difficult because prior history (in test) matters
  * Causes large explosion in state space
  * Exhaustive testing is unlikely to be feasible
* _Today:_ let's test out Chisel's `Queue` (stateful and uses `Decoupled`)

```scala
    Queue(UInt(n.W), numEntries, pipe=true, flow=false)
```

## Seq. Example - Queue - Model Implementation

* Be careful when modeling interactions with registers
  * Don't want register input to be available at register output too soon
* _Easy fix (for most of the time):_ ensure registers are read first in a cycle before written
* _Alternative:_ buffer register inputs and apply them all at once when cycle advances

In [ ]:
class QueueModel(numEntries: Int) {
    val mq = scala.collection.mutable.Queue[Int]()
    
    var deqReady = false
    def deqValid() = mq.nonEmpty
    // be sure to call attemptDeq before attemptEnq within a cycle
    def attemptDeq() = if (deqReady && deqValid) Some(mq.dequeue()) else None

    def enqReady() = mq.size < numEntries || (mq.size == numEntries && deqReady)
    // implies enqValid
    def attemptEnq(elem: Int): Unit = if (enqReady()) mq += elem
}

## Seq. Example - Queue - Model Demo Small

In [ ]:
val qm = new QueueModel(2)

// attempt push 1
qm.enqReady()
qm.deqValid()
qm.deqReady = false
qm.attemptDeq()
qm.attemptEnq(1)

// attempt push 2 & pop 1
qm.enqReady()
qm.deqValid()
qm.deqReady = true
qm.attemptDeq()
qm.attemptEnq(2)

## Seq. Example - Queue - Model Demo Long

In [ ]:
val qm = new QueueModel(2)

for (i <- 1 to 6) {
    qm.deqReady = i > 3
    print(s"deqV: ${qm.deqValid()}\tdeqR: ${qm.deqReady}\tdeqB: ${qm.attemptDeq()}")
    println(s"\tenqV: true\tenqR: ${qm.enqReady()}\tenqD: $i")
    qm.attemptEnq(i)
}

## Seq. Example - Queue - Manually Comparing to Model

In [ ]:
test(new Queue(UInt(32.W), 2, pipe=true, flow=false)) { dut =>
    val qm = new QueueModel(2)
    // always dequeue (for this example)
    qm.deqReady = true
    dut.io.deq.ready.poke(qm.deqReady.B)

    // try to dequeue on empty
    dut.io.deq.valid.expect(qm.deqValid.B)
    val deqResult0 = qm.attemptDeq()
    if (deqResult0.isDefined) dut.io.deq.bits.expect(deqResult0.get.U)
    dut.io.enq.ready.expect(qm.enqReady.B)
    dut.io.enq.valid.poke(false.B)
    dut.io.enq.bits.poke(0.U)
    dut.clock.step()
    
    // enqueue 1
    dut.io.deq.valid.expect(qm.deqValid.B)
    val deqResult1 = qm.attemptDeq()
    if (deqResult1.isDefined) dut.io.deq.bits.expect(deqResult1.get.U)
    dut.io.enq.ready.expect(qm.enqReady.B)
    dut.io.enq.valid.poke(true.B)
    dut.io.enq.bits.poke(1.U)
    qm.attemptEnq(1)
    dut.clock.step()
    
    // enqueue nothing, dequeue 1
    dut.io.deq.valid.expect(qm.deqValid.B)
    val deqResult2 = qm.attemptDeq()
    if (deqResult2.isDefined) dut.io.deq.bits.expect(deqResult2.get.U)
    dut.io.enq.ready.expect(qm.enqReady.B)
    dut.io.enq.valid.poke(false.B)
    dut.io.enq.bits.poke(0.U)
    dut.clock.step()
}

## Seq. Example - Queue - Automate Interaction

In [ ]:
def simCycle(dut: Queue[UInt], qm: QueueModel, enqValid: Boolean, deqReady: Boolean, enqData: Int=0) {
    qm.deqReady = deqReady
    dut.io.deq.ready.poke(qm.deqReady.B)
    dut.io.deq.valid.expect(qm.deqValid.B)
    val deqResult = qm.attemptDeq()
    if (deqResult.isDefined)
        dut.io.deq.bits.expect(deqResult.get.U)
    dut.io.enq.ready.expect(qm.enqReady.B)
    dut.io.enq.valid.poke(enqValid.B)
    dut.io.enq.bits.poke(enqData.U)
    if (enqValid)
        qm.attemptEnq(enqData)
    dut.clock.step()
    println(qm.mq)
}

test(new Queue(UInt(32.W), 2, pipe=true, flow=false)) { dut =>
    val qm = new QueueModel(2)
    simCycle(dut, qm, false, false)
    simCycle(dut, qm, true, false, 1)
    simCycle(dut, qm, false, true)
}

## Seq. Example - Queue - Test Fill & Drain

In [ ]:
def testFillAndDrain(numEntries: Int, n: Int) {
    test(new Queue(UInt(n.W), numEntries, pipe=true, flow=false)) { dut =>
        val qm = new QueueModel(numEntries)
        for (x <- 1 to numEntries+1) {  // fill
            simCycle(dut, qm, true, false, x)
        }
        for (x <- 1 to numEntries+1) {  // drain
            simCycle(dut, qm, false, true)
        }
    }
}

testFillAndDrain(2, 32)

## Seq. Example - Queue - Test Randomly

In [ ]:
def testRandomly(numEntries: Int, n: Int, numTrials: Int) {
    test(new Queue(UInt(n.W), numEntries, pipe=true, flow=false)) { dut =>
        val qm = new QueueModel(numEntries)
        for (i <- 1 until numTrials) {
            val tryEnq = scala.util.Random.nextBoolean
            val tryDeq = scala.util.Random.nextBoolean
            simCycle(dut, qm, tryEnq, tryDeq, i)
        }
    }
}

testRandomly(2, 32, 5)